<a href="https://colab.research.google.com/github/HectorENevarez/guidelines_detection/blob/main/Training_Model/face_mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Mask Detection Model
This ipynb goes over the steps taken to train the face detection model. <br>
This model is an adaptation of [Google's BigTransfer(BiT)](https://colab.research.google.com/github/google-research/big_transfer/blob/master/colabs/big_transfer_tf2.ipynb)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import tensorflow_datasets as tfds

import time

from PIL import Image
import requests
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

import os
import pathlib
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
!wget https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt

imagenet_int_to_str = {}

with open('ilsvrc2012_wordnet_lemmas.txt', 'r') as f:
  for i in range(1000):
    row = f.readline()
    row = row.rstrip()
    imagenet_int_to_str.update({i: row})

--2021-01-15 00:57:05--  https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 172.217.7.208, 142.250.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21675 (21K) [text/plain]
Saving to: ‘ilsvrc2012_wordnet_lemmas.txt.3’

ilsvrc2012_wordnet_ 100%[===================>]  21.17K  --.-KB/s    in 0s      

2021-01-15 00:57:06 (88.5 MB/s) - ‘ilsvrc2012_wordnet_lemmas.txt.3’ saved [21675/21675]



In [ ]:
tf_labels = ['No Mask', 'Mask', 'Unkown']

In [ ]:
model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
module = hub.KerasLayer(model_url)

## Creating the dataset
The following code was used in order to create the dataset of face masks. Each cell is for each class. In this case there would be three cells; Mask, no mask, unknown

In [ ]:
# Face Folder

face_img_loc = './drive/MyDrive/alt-faces-or-masks/faces' #Insert your path here
counter = 0
labels = []
mask_data = []
for img in os.listdir(face_img_loc):
  pic = cv2.imread(os.path.join(face_img_loc,img))
  pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
  pic = cv2.resize(pic,(32,32))
  mask_data.append(pic)
  labels.append(0) #Face

In [ ]:
#Mask Folder

mask_img_loc = './drive/MyDrive/alt-faces-or-masks/masks' # Insert path here
counter = 0
for img in os.listdir(mask_img_loc):
    pic = cv2.imread(os.path.join(mask_img_loc,img))
    pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
    pic = cv2.resize(pic,(32,32))
    mask_data.append(pic)
    labels.append(1) #Mask

In [ ]:
# Unknown folder

rec_img_loc = './drive/MyDrive/alt-faces-or-masks/reclassify' #Insert path here

for img in os.listdir(rec_img_loc):
    pic = cv2.imread(os.path.join(rec_img_loc,img))
    pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
    pic = cv2.resize(pic,(32,32))
    mask_data.append(pic)
    labels.append(2) #Unkown

In [ ]:
len(mask_data) # Number of images in dataset

2999

In [ ]:
#Shuffle dataset randomly

from sklearn.utils import shuffle

X, y = shuffle(mask_data, labels, random_state=0)

In [ ]:
y = np.array(y).astype('int64')

ds = tf.data.Dataset.from_tensor_slices(
    {"image" : X,
     "label" : y}) # Create data set

In [ ]:
ds

<TensorSliceDataset shapes: {image: (32, 32, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>

In [ ]:
# Seperation for train test split

train_split = 0.9
num_examples = 2999
num_train = int(train_split * num_examples)

ds_train = ds.take(num_train)
ds_test = ds.skip(num_train)

DATASET_NUM_TRAIN_EXAMPLES = num_examples

In [ ]:
print("The dataset has {} training samples and {} testing sample".format(len(ds_train), len(ds_test)))

The dataset has 2699 training samples and 300 testing sample


In [ ]:
class MyBiTModel(tf.keras.Model):
  """BiT with a new head."""

  def __init__(self, num_classes, module):
    super().__init__()

    self.num_classes = num_classes
    self.head = tf.keras.layers.Dense(num_classes, kernel_initializer='zeros')
    self.bit_model = module
  
  def call(self, images):
    # No need to cut head off since we are using feature extractor model
    bit_embedding = self.bit_model(images)
    return self.head(bit_embedding)

model = MyBiTModel(num_classes=3, module=module)

In [ ]:
IMAGE_SIZE = "=\u003C96x96 px" #@param ["=<96x96 px","> 96 x 96 px"]
DATASET_SIZE = "\u003C20k examples" #@param ["<20k examples", "20k-500k examples", ">500k examples"]

if IMAGE_SIZE == "=<96x96 px":
  RESIZE_TO = 160
  CROP_TO = 128
else:
  RESIZE_TO = 512
  CROP_TO = 480

if DATASET_SIZE == "<20k examples":
  SCHEDULE_LENGTH = 500
  SCHEDULE_BOUNDARIES = [200, 300, 400]
elif DATASET_SIZE == "20k-500k examples":
  SCHEDULE_LENGTH = 10000
  SCHEDULE_BOUNDARIES = [3000, 6000, 9000]
else:
  SCHEDULE_LENGTH = 20000
  SCHEDULE_BOUNDARIES = [6000, 12000, 18000]

In [ ]:
# Preprocessing helper functions

# Create data pipelines for training and testing:
BATCH_SIZE = 512
SCHEDULE_LENGTH = SCHEDULE_LENGTH * 512 / BATCH_SIZE

STEPS_PER_EPOCH = 10

def cast_to_tuple(features):
  return (features['image'], features['label'])
  
def preprocess_train(features):
  # Apply random crops and horizontal flips for all tasks 
  # except those for which cropping or flipping destroys the label semantics
  # (e.g. predict orientation of an object)
  features['image'] = tf.image.random_flip_left_right(features['image'])
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.image.random_crop(features['image'], [CROP_TO, CROP_TO, 3])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

def preprocess_test(features):
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

pipeline_train = (ds_train
                  .shuffle(10000)
                  .repeat(int(SCHEDULE_LENGTH * BATCH_SIZE / DATASET_NUM_TRAIN_EXAMPLES * STEPS_PER_EPOCH) + 1 + 50)  # repeat dataset_size / num_steps
                  .map(preprocess_train, num_parallel_calls=8)
                  .batch(BATCH_SIZE)
                  .map(cast_to_tuple)  # for keras model.fit
                  .prefetch(2))

pipeline_test = (ds_test.map(preprocess_test, num_parallel_calls=1)
                  .map(cast_to_tuple)  # for keras model.fit
                  .batch(BATCH_SIZE)
                  .prefetch(2))

In [ ]:
pipeline_train

<PrefetchDataset shapes: ((None, 128, 128, 3), (None,)), types: (tf.float32, tf.int64)>

In [ ]:
# Define optimiser and loss

lr = 0.003 * BATCH_SIZE / 512 

# Decay learning rate by a factor of 10 at SCHEDULE_BOUNDARIES.
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=SCHEDULE_BOUNDARIES, 
                                                                   values=[lr, lr*0.1, lr*0.001, lr*0.0001])
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

# Fine-tune model
history = model.fit(
    pipeline_train,
    batch_size=BATCH_SIZE,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=50,
    validation_data=pipeline_test
)

Epoch 1/50
10/10 [==============================] - 30s 1s/step - loss: 2.0419 - accuracy: 0.4688 - val_loss: 3.9154 - val_accuracy: 0.6267
Epoch 2/50
10/10 [==============================] - 10s 1s/step - loss: 1.7922 - accuracy: 0.7799 - val_loss: 2.4480 - val_accuracy: 0.7800
Epoch 3/50
10/10 [==============================] - 10s 1s/step - loss: 1.1809 - accuracy: 0.8507 - val_loss: 1.5704 - val_accuracy: 0.8033
Epoch 4/50
10/10 [==============================] - 10s 1s/step - loss: 0.8674 - accuracy: 0.8758 - val_loss: 1.2902 - val_accuracy: 0.8233
Epoch 5/50
10/10 [==============================] - 10s 1s/step - loss: 0.5919 - accuracy: 0.8889 - val_loss: 1.0184 - val_accuracy: 0.8333
Epoch 6/50
10/10 [==============================] - 10s 1s/step - loss: 0.5319 - accuracy: 0.8864 - val_loss: 0.9110 - val_accuracy: 0.8267
Epoch 7/50
10/10 [==============================] - 10s 1s/step - loss: 0.4647 - accuracy: 0.8964 - val_loss: 0.9101 - val_accuracy: 0.8133
Epoch 8/50
10/10 [==

In [ ]:
model.save('./drive/MyDrive/mask_Detect')

INFO:tensorflow:Assets written to: ./drive/MyDrive/mask_Detect/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/mask_Detect/assets
